In [37]:
# @title Simple Example: Conditional GAN
'''
We'll use a simplified version of a Conditional GAN that generates handwritten digits based on labels (0–9).
'''

"\nWe'll use a simplified version of a Conditional GAN that generates handwritten digits based on labels (0–9).\n"

In [38]:
# @title Step 1: Import required Libraries
import numpy as np
import matplotlib.pyplot as plt

# Import TensorFlow and Keras modules
import tensorflow as tf
from tensorflow.keras.datasets import mnist # Preloaded dataset of handwritten digits (0-9)
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, Concatenate
from tensorflow.keras.models import Model

In [39]:
# @title Step 2: Load and Normalize the Dataset

# Load MNIST dataset (images and labels)
(X_train, y_train), (_, _) = mnist.load_data()

# Normalize pixel values to range [-1,1] for better GAN performance
X_train = (X_train.astype(np.float32) - 127.5) / 127.5

# Flatten images from 28x28 to 784 vectors
X_train = X_train.reshape(X_train.shape[0], -1)

print("Training samples", X_train.shape[0])

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training samples 60000


In [40]:
# @title Step 3: Define Dimensions and Hyperparamters
latent_dim = 100 # Size of random noise vector
num_classes = 10 # Digits 0-9
img_shape = (784,) # Flattened image shap

In [41]:
# @title Step 4: Build the Generator

# Generator: Takes noise + label as input, ouput fake image

# Noise input
noise_input = Input(shape = (latent_dim,))
# Label Input
label_input = Input(shape=(1,), dtype = 'int32')

# Convert label to one-hot vector
label_embedding = tf.keras.layers.Embedding(num_classes, 50)(label_input)
label_embedding = Flatten()(label_embedding)

# Concatenate noise and label vector
merged_input = Concatenate()([noise_input,label_embedding])

# Dense layers to generate image
x = Dense(128, activation='relu')(merged_input)
x = Dense(256, activation='relu')(x)
x = Dense(np.prod(img_shape), activation='tanh')(x)

# Reshape back to image
generated_image = Reshape(img_shape)(x)

# Define generator model
generator = Model([noise_input, label_input], generated_image)
generator.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 1, 50)     │        500 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 50)        │          0 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 150)       │          0 │ input_layer_1[0]… │
│ (Concatenate)       │                   │            │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     19,328 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 256)       │     33,024 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 784)       │    201,488 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 784)       │          0 │ dense_3[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 254,340 (993.52 KB)

 Trainable params: 254,340 (993.52 KB)

 Non-trainable params: 0 (0.00 B)

In [42]:
# @title Step 5: Build the Discriminator

# Discriminator: Takes image + label and outputs real/fake image

# Image input
img_input = Input(shape = img_shape)
# Label Input
label_input = Input(shape=(1,), dtype = 'int32') # a single integer per sample

# Convert label to one-hot embedding
label_embedding = tf.keras.layers.Embedding(num_classes, 50)(label_input)
label_embedding = Flatten()(label_embedding)

# Concatenate Image and Label
merged_input = Concatenate()([img_input,label_embedding])

# Desnse Layers for Classification
x = Dense(256, activation = 'relu')(merged_input)
x = Dense(128, activation = 'relu')(x)
x = Dense(1, activation = 'sigmoid')(x) # Output probablity (real or fake)

# Define discriminator model
discriminator = Model([img_input, label_input], x)

# Compile Discriminator
discriminator.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
discriminator.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 1, 50)     │        500 │ input_layer_4[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 784)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 50)        │          0 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 834)       │          0 │ input_layer_3[0]… │
│ (Concatenate)       │                   │            │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 256)       │    213,760 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 128)       │     32,896 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 1)         │        129 │ dense_5[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 247,285 (965.96 KB)

 Trainable params: 247,285 (965.96 KB)

 Non-trainable params: 0 (0.00 B)

In [43]:
# @title Step 6: Discriminator during generator training
discriminator.trainable = False

# Connect generator + discriminator
noise = Input(shape =(latent_dim,))
label = Input(shape=(1,), dtype = 'int32')

# Generate Image
generated_img = generator([noise,label])

# Discriminate Image
validity = discriminator([generated_img, label])

# Define combined CGAN model
# CGAN model is used to train the generator using discriminator feedback:
cgan = Model([noise, label], validity)
cgan.compile(loss = 'binary_crossentropy', optimizer = 'adam')
cgan.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_6       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_1        │ (None, 784)       │    254,340 │ input_layer_5[0]… │
│ (Functional)        │                   │            │ input_layer_6[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_2        │ (None, 1)         │    247,285 │ functional_1[0][… │
│ (Functional)        │                   │            │ input_layer_6[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 501,625 (1.91 MB)

 Trainable params: 254,340 (993.52 KB)

 Non-trainable params: 247,285 (965.96 KB)

In [44]:
# @title Step 7 - Training Loop
epochs = 5000
batch_size = 128

for epoch in range(epochs):

  # Training Discriminiator

  # Select the random batch of real images
  idx = np.random.randint(0, X_train.shape[0], batch_size)
  real_imgs = X_train[idx]
  labels = y_train[idx]

  # Generate fake images
  noise = np.random.normal(0, 1, (batch_size, latent_dim))
  gen_imgs = generator.predict([noise, labels])


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━

KeyboardInterrupt: 

In [ ]:
# Generate sample images for a specific label (e.g., '7')
label = np.array([[7]])
noise = np.random.normal(0, 1, (1, latent_dim))

# Generate fake image for label '7'
gen_img = generator.predict([noise, label])[0]

# Reshape and display
plt.imshow(gen_img.reshape(28, 28), cmap='gray')
plt.title("Generated Digit: 7")
plt.axis('off')
plt.show()